## Các lớp dữ liệu

In [1]:
class Answer:
    '''Lớp đáp án:
    text - (str) - Nội dung đáp án
    flag - (bool) - đáp án đúng hay sai
    '''
    def __init__(self, text, flag):
        self.text = text
        self.flag = flag
    
    def show(self):
        print(f'{self.text} - {self.flag}')
        
    def clone(self):
        return Answer(self.text, False)

In [2]:
class Question:
    '''Lớp câu hỏi:
    text - (str): Nội dung câu hỏi
    answers - (list): 4 đáp án của câu hỏi
    '''
    def __init__(self, text, answers):
        self.text = text
        self.answers = answers
    
    def show(self):
        print(self.text)
        for i in self.answers:
            i.show()
        
    def not_answered(self):
        result = True
        for i in self.answers:
            if i.flag == True:
                result = False
                break
        return result
    
    def correct(self, q):
        result = True
        for i in range(len(self.answers)):
            if self.answers[i].flag != q.answers[i].flag:
                result = False
                break
        return result
            
    def clone(self):
        ans_copy = []
        for an in self.answers:
            ans_copy.append(an.clone())
        return Question(self.text, ans_copy)

In [3]:
text = "Quốc gia nào sau đây thuộc châu Mỹ?"
ans1 = Answer('Trung Quốc', False)
ans2 = Answer('Mexico', True)
ans3 = Answer('Argentina', True)
ans4 = Answer('Iran', False)
da = [ans1, ans2, ans3, ans4]
question = Question(text, da)
question.show()

Quốc gia nào sau đây thuộc châu Mỹ?
Trung Quốc - False
Mexico - True
Argentina - True
Iran - False


In [4]:
class Test:
    """Lớp test:
    qs - (list(Question)) - một list câu hỏi
    """
    def __init__(self, qs):
        self.qs = qs
        
    def clone(self):
        copy_test = []
        for q in self.qs:
            copy_test.append(q.clone())
        return Test(copy_test)
        
    def compare(self, t):
        '''Hàm so sánh hai test: self - t'''
        correct = 0
        total = len(self.qs)
        answered = 0
        none_answered = 0
        for i in range(len(self.qs)):
            if self.qs[i].not_answered():
                none_answered+=1
            if self.qs[i].correct(t.qs[i]):
                correct+=1
        return total, total-none_answered, correct
        # Câu chưa trả lời là câu như thế nào
        # là câu các đáp án là False
        # Câu đúng là câu như thế nào
    
    def loadQuestion(self, path):
        f= open(path,'r', encoding='utf8')
        for line in f:
            items=line.split('|')
            answer1=Answer(items[1], True if items[5]=='True' else False)
            answer2=Answer(items[2], True if items[6]=='True' else False)
            answer3=Answer(items[3], True if items[7]=='True' else False)
            answer4=Answer(items[4], True if items[8]=='True' else False)
            ans = [answer1, answer2, answer3, answer4]
            q=Question(items[0], ans)
            self.qs.append(q)
            
    def saveQuestion(self,path):
        f= open(path,'w', encoding='utf8')
        for q in self.qs:
            f.write(q.text)
            f.write('|')
            f.write(q.answers[0].text)
            f.write('|')
            f.write(q.answers[1].text)
            f.write('|')
            f.write(q.answers[2].text)
            f.write('|')
            f.write(q.answers[3].text)
            f.write('|')
            f.write(str(q.answers[0].flag))
            f.write('|')
            f.write(str(q.answers[1].flag))
            f.write('|')
            f.write(str(q.answers[2].flag))
            f.write('|')
            f.write(str(q.answers[3].flag))
            f.write('\n')
            
    def createThreeQuestions(self):
        ans1 = Answer('Trung Quốc', False)
        ans2 = Answer('Mexico', True)
        ans3 = Answer('Argentina', True)
        ans4 = Answer('Iran', False)
        da = [ans1, ans2, ans3, ans4]
        q1 = Question("Quốc gia nào sau đây thuộc châu Mỹ?", da)
        
        answer5 = Answer('Lào', False)
        answer6 = Answer('Việt Nam', True)
        answer7 = Answer('Malaysia', False)
        answer8 = Answer('Thái Lan', False)
        ans2 = [answer5, answer6, answer7, answer8]
        q2 = Question('Hà Nội là thủ đô nước nào? ', ans2)
        
        answer10 = Answer('Việt Nam', True)
        answer20 = Answer('Mexico', False)
        answer30 = Answer('Pháp', False)
        answer40 = Answer('Singapore', True) 
        ans3 = [answer10, answer20, answer30, answer40]
        q3 = Question('Nước nào rộng nhất thế giới? ', ans3)
        
        self.qs = [q1, q2, q3]

In [5]:
from tkinter import *

root = Tk()
root.title('Hệ thống trắc nghiệm')
root.geometry('710x580')
root.resizable(False,False)
lq= []
index = 0
original = Test(lq)
original.loadQuestion('data.txt')
finished = False

copy_test = original.clone()
# Tạo ra 2 bản list câu hỏi
# 1 bản là bản gốc dùng để đối chiếu
# 1 bản sao, các đáp án ban đầu đều là false - người dùng chưa trả lời

def goto_next_question(event):
    global index
    if index < len(copy_test.qs)-1:
        index +=1
        btn_previous['state'] = 'normal'
    if index==len(copy_test.qs)-1:
        btn_next['state'] = 'disabled'
    displayQuestion(index)
    
def goto_previous_question(event):
    global index
    if index>0:
        index -=1
        btn_next['state'] = 'normal'
    if index==0:
        btn_previous['state'] = 'disabled'
    displayQuestion(index)

    
def displayQuestion(index):
    global finished
    # Hiển thị nội dung câu hỏi
    lb_frame_question['text'] = f'Câu hỏi thứ {index+1}'
    question_text.delete(1.0, END)
    question_text.insert(1.0, copy_test.qs[index].text)    
    #Hiển thị nội dung đáp án
    text_answer1.delete(1.0, END)
    text_answer1.insert(1.0, copy_test.qs[index].answers[0].text)  
    cb1.set(copy_test.qs[index].answers[0].flag)
    
    text_answer2.delete(1.0, END)
    text_answer2.insert(1.0, copy_test.qs[index].answers[1].text)
    cb2.set(copy_test.qs[index].answers[1].flag)
    
    text_answer3.delete(1.0, END)
    text_answer3.insert(1.0, copy_test.qs[index].answers[2].text)   
    cb3.set(copy_test.qs[index].answers[2].flag)
    
    text_answer4.delete(1.0, END)
    text_answer4.insert(1.0, copy_test.qs[index].answers[3].text)   
    cb4.set(copy_test.qs[index].answers[3].flag)
    if finished:
        ckbt1['state'] = 'disabled'
        ckbt2['state'] = 'disabled'
        ckbt3['state'] = 'disabled'
        ckbt4['state'] = 'disabled'
        # Clear color
        ckbt1['bg'] = 'white'
        ckbt2['bg'] = 'white'
        ckbt3['bg'] = 'white'
        ckbt4['bg'] = 'white'
        
        text_answer1['bg'] = 'white'
        text_answer2['bg'] = 'white'
        text_answer3['bg'] = 'white'
        text_answer4['bg'] = 'white'
        
        if copy_test.qs[index].correct(original.qs[index]):
            # câu hỏi này người dùng trả lời đúng
            for j in range(len(copy_test.qs[index].answers)):
                if copy_test.qs[index].answers[j].flag:
                    if j==0:
                        ckbt1['bg'] = 'green'
                        text_answer1['bg'] = 'green'
                    elif j==1:
                        ckbt2['bg'] = 'green'
                        text_answer2['bg'] = 'green'
                    elif j==2:
                        ckbt3['bg'] = 'green'
                        text_answer3['bg'] = 'green'
                    else:
                        ckbt4['bg'] = 'green'
                        text_answer4['bg'] = 'green'
        else:
            # sai
            for j in range(len(copy_test.qs[index].answers)):
                if copy_test.qs[index].answers[j].flag:
                    if j==0:
                        ckbt1['bg'] = 'red'
                        text_answer1['bg'] = 'red'
                    elif j==1:
                        ckbt2['bg'] = 'red'
                        text_answer2['bg'] = 'red'
                    elif j==2:
                        ckbt3['bg'] = 'red'
                        text_answer3['bg'] = 'red'
                    else:
                        ckbt4['bg'] = 'red'
                        text_answer4['bg'] = 'red'
    
def get_temporary_answer1():
    global index
    copy_test.qs[index].answers[0].flag = cb1.get()

def get_temporary_answer2():
    global index
    copy_test.qs[index].answers[1].flag = cb2.get()

def get_temporary_answer3():
    global index
    copy_test.qs[index].answers[2].flag = cb3.get()

def get_temporary_answer4():
    global index
    copy_test.qs[index].answers[3].flag = cb4.get()


def finish(event):
    global finished
    finished = True
    total, answered, correct = copy_test.compare(original)
    lb_result['text'] = f'Tổng số câu hỏi: {total}, trả lời {answered}, không trả lời {total-answered}, trả lời đúng {correct}, trả lời sai {total-correct}, đạt {round(correct*100/total, 2)}%' 
    
def insert(event):
    answer1 = Answer(text_answer1.get(1.0, END)[:-1],cb1.get())
    answer2 = Answer(text_answer2.get(1.0, END)[:-1],cb2.get())
    answer3 = Answer(text_answer3.get(1.0, END)[:-1],cb3.get())
    answer4 = Answer(text_answer4.get(1.0, END)[:-1],cb4.get())
    ans = [answer1, answer2, answer3, answer4]
    q=Question(question_text.get(1.0,END)[:-1], ans)
    original.qs.append(q)
    original.saveQuestion('data.txt')
    
    
myfont = ('Helvetica', 12)
lb_frame_question = LabelFrame(root, text="Câu hỏi 1", font=myfont, fg='blue', height=10)
lb_frame_question.pack(fill=BOTH, expand=True, side=TOP)

question_text = Text(lb_frame_question, font=myfont, height=8)
question_text.pack(padx=5, pady=5, ipadx=5)

lb_frame_answers = LabelFrame(root, text="Các đáp án", font=myfont, fg='blue', height=50)
lb_frame_answers.pack(fill=BOTH, expand=True)

cb1 = BooleanVar()
cb1.set(0)
cb2 = BooleanVar()
cb2.set(0)
cb3 = BooleanVar()
cb3.set(0)
cb4 = BooleanVar()
cb4.set(0)

ckbt1 = Checkbutton(lb_frame_answers, text='',variable=cb1, onvalue=True, offvalue=False, command=get_temporary_answer1)
ckbt1.grid(column=0, row=0)
ckbt2 = Checkbutton(lb_frame_answers, text='',variable=cb2, onvalue=True, offvalue=False, command=get_temporary_answer2)
ckbt2.grid(column=2, row=0)
ckbt3 = Checkbutton(lb_frame_answers, text='',variable=cb3, onvalue=True, offvalue=False, command=get_temporary_answer3)
ckbt3.grid(column=0, row=1)
ckbt4 = Checkbutton(lb_frame_answers, text='',variable=cb4, onvalue=True, offvalue=False, command=get_temporary_answer4)
ckbt4.grid(column=2, row=1)


text_answer1 = Text(lb_frame_answers, font=myfont, width=35, height=7)
text_answer1.grid(column=1, row=0)
text_answer2 = Text(lb_frame_answers, font=myfont, width=35, height=7)
text_answer2.grid(column=3, row=0)
text_answer3 = Text(lb_frame_answers, font=myfont, width=35, height=7)
text_answer3.grid(column=1, row=1)
text_answer4 = Text(lb_frame_answers, font=myfont, width=35, height=7)
text_answer4.grid(column=3, row=1)

lb_frame_actions = LabelFrame(root, text="Các thao tác", font=myfont, fg='blue', height=50)
lb_frame_actions.pack(fill=BOTH, expand=True)

photo_previous = PhotoImage(file='images/previous.png')
btn_previous = Button(lb_frame_actions, text='Câu hỏi trước',fg='blue', font=('Helvetica', 12, 'bold'), image=photo_previous, compound=LEFT)
btn_previous.grid(column=0,row=0, pady=5)
btn_previous.bind('<Button-1>', goto_previous_question)

photo_next = PhotoImage(file='images/next.png')
btn_next = Button(lb_frame_actions, text='Câu hỏi tiếp',fg='blue', font=('Helvetica', 12, 'bold'), image=photo_next, compound=RIGHT)
btn_next.grid(column=1,row=0,pady=5)
btn_next.bind('<Button-1>', goto_next_question)

photo_finish = PhotoImage(file='images/finish.png')
btn_confirm = Button(lb_frame_actions, text='Kết thúc',fg='red', font=('Helvetica', 12, 'bold'),image=photo_finish, compound=LEFT)
btn_confirm.grid(column=19,row=0,pady=5)
btn_confirm.bind('<Button-1>', finish)

btn_insert = Button(lb_frame_actions, text='Thêm câu hỏi',fg='red', font=('Helvetica', 12, 'bold'),image=photo_finish, compound=LEFT)
btn_insert.grid(column=20,row=0,pady=5)
btn_insert.bind('<Button-1>', insert)

#text = Tổng số câu hỏi: 30, trả lời 30, không trả lời 0, trả lời đúng 20, trả lời sai 10, đạt 66% 
lb_result = Label(lb_frame_actions,font= myfont,fg='green', text=' ')
lb_result.grid(column=0,row=1, columnspan=20, pady=10)

displayQuestion(0)
root.mainloop()